In [2]:
import requests

In [3]:
# Assigning the value to the Variable
base_url = 'https://nominatim.openstreetmap.org/search?'

In [4]:
# Creating a Parameter Dictionary
params = {
       'format':'json',
       'q': 'Eiffel tower'
   }

In [5]:
result = requests.get(base_url,params=params)

In [6]:
result.status_code

200

In [7]:
# Since the Status Code is 200 It represents Sucessful. So get the data into a JSON format
result.json()

[{'place_id': 257507700,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 5013364,
  'boundingbox': ['48.8574753', '48.8590465', '2.2933084', '2.2956897'],
  'lat': '48.8582602',
  'lon': '2.29449905431968',
  'display_name': 'Tour Eiffel, 5, Avenue Anatole France, Gros-Caillou, 7e, Paris, Île-de-France, France métropolitaine, 75007, France',
  'class': 'tourism',
  'type': 'attraction',
  'importance': 0.653772102971417,
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_point_of_interest.p.20.png'},
 {'place_id': 28224069,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2689315641,
  'boundingbox': ['51.33355', '51.33365', '-116.23505', '-116.23495'],
  'lat': '51.3336',
  'lon': '-116.235',
  'display_name': 'Eiffel Tower, Alberta, Canada',
  'class': 'natural',
  'type': 'peak',
  'importance': 0.5,
  'icon': 'https://nomina

In [8]:
params = {
    'format' : "json",
    'q' : "Cair Paravel, Narnia",
    'limit' : 1
}

In [9]:
requests.get(base_url,  params=params).json()

[]

In [10]:
from time import sleep

def nomination_geocode(address, format='json', limit=1, **kwargs):
    params = {
        'q' : address,
        'format' : format,
        'limit' : limit,
        **kwargs
    }
    
    response = requests.get(base_url, params=params)
    response.raise_for_status() # It will raise exception if Request is unsuccesful.
    sleep(1)
    
    return response.json()

In [11]:
# Calling the Created Function
nomination_geocode('Eiffel tower')

[{'place_id': 257507700,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 5013364,
  'boundingbox': ['48.8574753', '48.8590465', '2.2933084', '2.2956897'],
  'lat': '48.8582602',
  'lon': '2.29449905431968',
  'display_name': 'Tour Eiffel, 5, Avenue Anatole France, Gros-Caillou, 7e, Paris, Île-de-France, France métropolitaine, 75007, France',
  'class': 'tourism',
  'type': 'attraction',
  'importance': 0.653772102971417,
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_point_of_interest.p.20.png'}]

In [12]:
# Again Calling Function with different Input Parameters
nomination_geocode(address=None, street='221B Baker Street', city="London", country="Great Britain")

[{'place_id': 260072193,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 176312585,
  'boundingbox': ['51.5237026', '51.5237936', '-0.158698', '-0.1584418'],
  'lat': '51.52374805',
  'lon': '-0.158569881653613',
  'display_name': '221B, Baker Street, Marylebone, City of Westminster, London, Greater London, England, NW1 6XE, UK',
  'class': 'building',
  'type': 'terrace',
  'importance': 0.5209999999999999}]

#### Reading and Writing Data To the CSV Files

In [13]:
from csv import DictReader, DictWriter
path = './cities.csv'

with open(path, 'r') as f:
    data = list(DictReader(f))

In [14]:
def read_csv(path):
    
    with open(path, 'r') as f:
        return list(DictReader(f))

In [15]:
def write_csv(data, path, mode='w'):
    '''write data to csv or append to existing one'''
    if mode not in 'wa':
        raise ValueError("mode should be either 'w' or 'a'")
    with open(path, mode) as f:
        writer = DictWriter(f, fieldnames=data[0].keys())
        if mode == 'w':
            writer.writeheader()
        for row in data:
            writer.writerow(row)

In [16]:
cities = read_csv('./cities.csv')

In [17]:
cities[0]

OrderedDict([('name', 'Tokio'), ('population', '38.05'), ('country', 'Japan')])

In [18]:
write_csv(cities, './my_cities.csv')

####  GeoCoding Addresses in batches

In [19]:
from tqdm import tqdm

def geocode_bulk(data, column='address', verbose=False):
    result, errors = [], []
    
    for row in tqdm(data):
        try:
            search = nomination_geocode(row[column], limit=1)
            if(len(search) == 0):
                result.append(row)
                if verbose:
                    print(f"Can't find anything for {row[column]}")
            else:
                info = search[0]
                info.update(row)
                result.append(info)
        except Exception as e:
            if verbose:
                print(e)
            row['error'] = e
            errors.append(row)
    
    if len(errors) > 0 and verbose:
        print(f"{len(errors)/len(data)} rows failed")
    
    return result, errors

In [20]:
result, errors = geocode_bulk(cities, column='name', verbose=True)

100%|██████████| 10/10 [00:22<00:00,  2.26s/it]


#### Moving Code to separate Module(Creating the Module)

In [21]:
'''
path_in = './cities.csv'
path_out = './geocoded.csv'
data = read_csv(path_in)
result, errors = geocode_bulk(data, column='address', verbose=True)
write_csv(result, path_out)

'''
# The Above code doesn't required actually.


"\npath_in = './cities.csv'\npath_out = './geocoded.csv'\ndata = read_csv(path_in)\nresult, errors = geocode_bulk(data, column='address', verbose=True)\nwrite_csv(result, path_out)\n\n"